In [1]:
import matplotlib.pyplot as plt

In [2]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#from skopt.space import Real, Categorical, Integer

from sklearn import preprocessing


In [3]:
def eulerPlant(Cx,Ca,Cp,F,Tsamp):
    #next states for plant
    #Define constants
    mumax =40
    Ks =2e-1
    Yx =0.04
    Yp =0.8
    pstar=1.8
    Ca0=10.0

    kd= 1e-2;
    aa = 5.5;
    bb = 8e-1;
    
        
    if Cp>pstar:
       mu = 0
    else:
       mu = mumax*(Ca/(Ca+Ks))*(1-Cp/pstar)**3

        

    #Rate Definitions
    fcx=(mu-kd)*Cx - F*Cx
    fcp=aa*fcx +bb*Cx- F*Cp
    fca=-(1/Yx)*fcx - (1/Yp)*fcp + F*(Ca0-Ca)
   
    Cx_kplusone = Cx + Tsamp*fcx
    Ca_kplusone = Ca + Tsamp*fca
    Cp_kplusone = Cp + Tsamp*fcp
    
    
    return Cx_kplusone, Ca_kplusone,Cp_kplusone

In [4]:
def eulerModel(Cx,Ca,Cp,F,Tsamp):
    #next states from model (original)
    #Define constants
    mumax =25.0#
    Ks =0.8
    Yx =0.05 #
    Yp =0.8 # 
    Ca0=10.0

    kd= 1e-13; #
    aa = 3.8;
    bb = 0.05;
    pstar=0.5

    np = 0.95;
    
    if Cp>pstar:
       mu = 0
    else:
       mu = mumax*(Ca/(Ca+Ks))*(1-Cp/pstar)


    #Rate Definitions
    fcx=(mu-kd)*Cx - F*Cx
    fcp=aa*fcx  - F*Cp
    fca=-(1/Yx)*fcx - (1/Yp)*fcp + F*(Ca0-Ca)
   
    Cx_kplusone = Cx + Tsamp*fcx
    Ca_kplusone = Ca + Tsamp*fca
    Cp_kplusone = Cp + Tsamp*fcp
    
    
    return Cx_kplusone, Ca_kplusone,Cp_kplusone

In [5]:
def ModelSimulator(Tsamp,Nsamp,F):
    """
    Returns a trajectory of the HF model given some flow rate sequence
    """
    tsize = len(F)+1
    Cx_model = np.zeros((tsize,1))
    Ca_model = np.zeros((tsize,1))
    Cp_model = np.zeros((tsize,1))
    Cx_model[0]=0.2;Ca_model[0]=15;Cp_model[0]=0

    for k in range(len(F)):
          Cx_model[k+1], Ca_model[k+1], Cp_model[k+1] = eulerModel(Cx_model[k],Ca_model[k],Cp_model[k],F[k],Tsamp)
    return Cx_model, Ca_model, Cp_model

In [6]:
def ResnetModelSimulator(Tsamp,Nsamp,F,DropModel):
    """
    Returns a trajectory of the HF model given some flow rate sequence
    """
    tsize = len(F)+1
    Cx_resnet = np.zeros((tsize,1))
    Ca_resnet = np.zeros((tsize,1))
    Cp_resnet = np.zeros((tsize,1))
    Cx_resnet[0]=0.2;Ca_resnet[0]=15;Cp_resnet[0]=0
    # xpr = np.array([x0,alpha[0],alpha[1],alpha[2],alpha[3],delta])
    #print(xpr)
    #xprn = xpr.reshape(1,6)
    #xnew = DropModel.predict(xprn)

    for k in range(len(F)):
        xpr = np.array([F[k],Cx_resnet[k],Ca_resnet[k],Cp_resnet[k]])
        xprn = xpr.reshape(1,4)
        #    xnew = DropModel.predict(xprn)
        futurep = DropModel.predict(xprn)
        Cx_resnet[k+1] = futurep[0][0]
        Ca_resnet[k+1] = futurep[0][1]
        Cp_resnet[k+1] = futurep[0][2]

    return Cx_resnet, Ca_resnet, Cp_resnet

In [7]:
def PlantSimulator(Tsamp,Nsamp,F):
    """
    Returns a trajectory of the HF model given some flow rate sequence
    """
    tsize = len(F)+1
    Cx_plant = np.zeros((tsize,1))
    Ca_plant = np.zeros((tsize,1))
    Cp_plant = np.zeros((tsize,1))
    Cx_plant[0]=0.2;Ca_plant[0]=15;Cp_plant[0]=0

    for k in range(len(F)):
          Cx_plant[k+1], Ca_plant[k+1], Cp_plant[k+1] = eulerPlant(Cx_plant[k],Ca_plant[k],Cp_plant[k],F[k],Tsamp)
    return Cx_plant, Ca_plant, Cp_plant

In [8]:
Nsamp=30; #Time steps
random.seed()
F=[]
F1 = np.random.uniform(low=5, high=15, size=(Nsamp,))

In [9]:
def HF_Model_Data(num,Nsamp): 

    random.seed(num)
    F=[]
    tarr=[]
    Tsamp = 0.005
    F = np.random.uniform(low=5, high=15, size=(Nsamp,))
    tarr = np.zeros((Nsamp,1))
    Cx_model,Ca_model,Cp_model = ModelSimulator(Tsamp,Nsamp,F)

    for j in range(Nsamp):
        tarr[j] = j*Tsamp

    tarr = np.append(tarr, np.repeat(np.nan, 1))
    F = np.append(F, np.repeat(np.nan, 1))
    return F,Cx_model,Ca_model,Cp_model

In [10]:
def iterative_model(num,Nsamp):
    
    Xdata =np.zeros((Nsamp,4))
    Ydata =np.zeros((Nsamp,3))
    F,Cx_model,Ca_model,Cp_model = HF_Model_Data(num,Nsamp)

    for i in range(len(F)-1):
        Xdata[i,:] = [F[i],Cx_model[i],Ca_model[i],Cp_model[i]]
        Ydata[i,:] = [Cx_model[i+1],Ca_model[i+1],Cp_model[i+1]]
    
    return Xdata, Ydata



In [11]:
Nruns=80;

X =np.zeros((Nruns,Nsamp,4))
Y =np.zeros((Nruns,Nsamp,3))

num = random.randint(1, 50000000)
random.seed(num)

for j in range(Nruns):
    X[j,:,:],Y[j,:,:]= iterative_model(num,Nsamp)


In [12]:
X.shape

(80, 30, 4)

In [13]:
Xdata = X.reshape(Nruns*Nsamp,4)  #[Flow Rate, Cx_past, Ca_past, Cp_past]


In [14]:
Xdnn = Xdata[0:70*Nsamp,:]

In [15]:
Xdnn.shape

(2100, 4)

In [16]:
Xval = Xdata[50*Nsamp:,:]

In [17]:
Xval.shape

(900, 4)

In [18]:
Ydata = Y.reshape(Nruns*Nsamp,3)  #[Ca_future, Cb_future, Tr_future]

In [19]:
Ydnn = Ydata[0:70*Nsamp,:]

In [20]:
Ydnn.shape

(2100, 3)

In [21]:
Yval = Ydata[50*Nsamp:,:]

In [22]:
Yval.shape

(900, 3)

In [23]:
def swish(x, beta=1):
    return x * sigmoid(beta * x)

In [24]:
def swish_eval(x):
    return x/(1+np.exp(-x))

In [25]:
def get_model_layers(nodes, layers, activation, learning_rate, training=None):
    if activation == 'swish':
        activation = swish
    else:
        activation = activation

    inputs = tf.keras.layers.Input(shape=(in_dim,))
  #  xadd = (np.array(inputs[:, :out_dim])).reshape(None,1)
   # print(inputs[:, :out_dim].shape)
    #print(xadd.shape)
    x = tf.keras.layers.Dense(units=nodes, activation=activation)(inputs)
    for i in range(layers - 1):  # at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation=activation)(x)

    #x = tf.keras.layers.Dropout(rate=0.1)(x, training=training)
    outputs = tf.keras.layers.Dense(out_dim, activation="linear")(x)
    outputs = tf.keras.layers.Add()([outputs, inputs[:, 1:]])

    #outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]])
    # outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]]) NN4UQ
    #    out = keras.layers.Dense(units=3, activation= "linear")(added)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error')
    return model

In [26]:
get_custom_objects().update({'swish': Activation(swish)})


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(Xdnn, Ydnn, shuffle=True, train_size=0.8, random_state=42)
in_dim = X_train.shape[1]
out_dim = Y_train.shape[1]

In [28]:
out_dim

3

In [29]:
np.max(X_test[:,0])

14.973745167335403

In [30]:
#print(X_train.shape)
learning_rate=0.05#gp_result.x[0]
num_layers = 5#gp_result.x[1]
num_nodes = 8 #gp_result.x[2]
batch_size=10#gp_result.x[3]
epochs=300
activation='swish'
DropModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                             activation=activation, learning_rate=learning_rate,
                             training=True)  # True-activates dropout during prediction
DropModel.summary()
DropModel.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data = (X_test, Y_test))

DetModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                            activation=activation, learning_rate=learning_rate)
#name = '{}_{}_{}_{}'.format(num_nodes, num_layers, activation, learning_rate)
#DropModel.save(root + 'results/nc/' + name)
DetModel.set_weights(DropModel.get_weights())
score = DetModel.evaluate(X_test, Y_test, verbose=0)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 8)            40          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            72          dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 8)            72          dense_1[0][0]                    
______________________________________________________________________________________________

In [59]:
'''
#Simulating both systems using some initial condition
#The predictions will be quite off since the difference in the models propagates within a trajectory
F=[]
tarr=[]
Tsamp = 0.005
Nsamp = 50
F = np.random.uniform(low=5, high=15, size=(Nsamp,))
tarr = np.zeros((Nsamp,1))

#Ca_plant,Cb_plant,Tr_plant,Tk_plant = PlantSimulator(Tsamp,Nsamp,F)
#Get states for a sequence of F, given the models
Cx_model,Ca_model,Cp_model = ModelSimulator(Tsamp,Nsamp,F1)
Cx_plant,Ca_plant,Cp_plant = PlantSimulator(Tsamp,Nsamp,F1)
Cx_resnet,Ca_resnet,Cp_resnet = ResnetModelSimulator(Tsamp,Nsamp,F1,DropModel)




for j in range(Nsamp):
    tarr[j] = j*Tsamp

tarr = np.append(tarr, np.repeat(np.nan, 1))
F = np.append(F, np.repeat(np.nan, 1))

plt.figure(dpi=50)
plt.plot(Cx_plant,'r--',label='Plant',linewidth=3.0,alpha=0.8)
plt.plot(Cx_model,'g--',label='HF Model',linewidth=6.0)
plt.plot(Cx_resnet,'b-',label='ResNET',linewidth=3.0)
plt.xlabel("Sampling step")
plt.ylabel("Biomass")
plt.legend()

plt.figure(dpi=50)
plt.plot(Ca_plant,'r--',label='Plant',linewidth=3.0,alpha=0.8)
plt.plot(Ca_model,'g--',label='HF Model',linewidth=6.0)
plt.plot(Ca_resnet,'b-',label='ResNET',linewidth=3.0)
plt.xlabel("Sampling step")
plt.ylabel("Acetate")
plt.legend()


plt.figure(dpi=50)
plt.plot(Cp_plant,'r--',label='Plant',linewidth=3.0,alpha=0.8)
plt.plot(Cp_model,'g--',label='HF Model',linewidth=6.0)
plt.plot(Cp_resnet,'b-',label='ResNET',linewidth=3.0)
plt.xlabel("Sampling step")
plt.ylabel("Hydrogen")
plt.legend()

plt.figure(dpi=50)
plt.plot(F1,linewidth=3.0)
plt.legend()
plt.xlabel("Sampling step")
plt.ylabel("Flow Rate")
plt.legend()
'''

'\n#Simulating both systems using some initial condition\n#The predictions will be quite off since the difference in the models propagates within a trajectory\nF=[]\ntarr=[]\nTsamp = 0.005\nNsamp = 50\nF = np.random.uniform(low=5, high=15, size=(Nsamp,))\ntarr = np.zeros((Nsamp,1))\n\n#Ca_plant,Cb_plant,Tr_plant,Tk_plant = PlantSimulator(Tsamp,Nsamp,F)\n#Get states for a sequence of F, given the models\nCx_model,Ca_model,Cp_model = ModelSimulator(Tsamp,Nsamp,F1)\nCx_plant,Ca_plant,Cp_plant = PlantSimulator(Tsamp,Nsamp,F1)\nCx_resnet,Ca_resnet,Cp_resnet = ResnetModelSimulator(Tsamp,Nsamp,F1,DropModel)\n\n\n\n\nfor j in range(Nsamp):\n    tarr[j] = j*Tsamp\n\ntarr = np.append(tarr, np.repeat(np.nan, 1))\nF = np.append(F, np.repeat(np.nan, 1))\n\nplt.figure(dpi=50)\nplt.plot(Cx_plant,\'r--\',label=\'Plant\',linewidth=3.0,alpha=0.8)\nplt.plot(Cx_model,\'g--\',label=\'HF Model\',linewidth=6.0)\nplt.plot(Cx_resnet,\'b-\',label=\'ResNET\',linewidth=3.0)\nplt.xlabel("Sampling step")\nplt.ylabe

Some preliminary work to extract weights; Ignore

The above are open-loop predictions

In [32]:
DropModel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 8)            40          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            72          dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 8)            72          dense_1[0][0]                    
______________________________________________________________________________________________

In [33]:
weights = DropModel.get_weights() # returs a numpy list of weights

In [34]:
weights

[array([[ 0.23999472, -0.04628917,  0.09226926, -0.5626267 , -0.11783998,
         -0.2238071 ,  0.3168612 , -0.3618773 ],
        [-0.4343271 , -0.07948402, -0.01344697, -0.12236922,  0.9121022 ,
          0.4724184 ,  0.37494585,  0.17856601],
        [-0.4246694 ,  0.4687562 ,  0.1408947 , -0.51426244, -0.04461471,
          0.4588363 , -0.5802143 , -0.26814318],
        [-0.07323387,  1.2612872 , -2.2421353 ,  0.33727244, -0.7457813 ,
         -2.0778406 , -0.405954  ,  0.16835864]], dtype=float32),
 array([ 0.17453043, -0.11173429, -0.06356671, -0.02668981,  0.48467213,
        -0.17731553,  0.077495  ,  0.06343799], dtype=float32),
 array([[ 0.07478299,  0.28099123, -0.38207337, -0.6469538 ,  0.16221142,
          0.02425827,  0.17009296, -0.63206214],
        [ 0.5018329 , -0.17502545,  0.00568105, -0.61883855, -0.45498002,
         -0.4087016 , -0.5520174 , -0.15879826],
        [-0.17655648,  0.5888297 ,  0.36615887, -0.27345115,  0.48871818,
         -0.4265416 , -0.1736504 ,

In [35]:
uk=np.array([10.0]) #Control Input
xk=np.array([0.2,15,0]) #Current States
zk=np.concatenate((uk,xk)).reshape(1,4) #

In [36]:
Wmat = np.array(weights)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [37]:
Wmat[0]

array([[ 0.23999472, -0.04628917,  0.09226926, -0.5626267 , -0.11783998,
        -0.2238071 ,  0.3168612 , -0.3618773 ],
       [-0.4343271 , -0.07948402, -0.01344697, -0.12236922,  0.9121022 ,
         0.4724184 ,  0.37494585,  0.17856601],
       [-0.4246694 ,  0.4687562 ,  0.1408947 , -0.51426244, -0.04461471,
         0.4588363 , -0.5802143 , -0.26814318],
       [-0.07323387,  1.2612872 , -2.2421353 ,  0.33727244, -0.7457813 ,
        -2.0778406 , -0.405954  ,  0.16835864]], dtype=float32)

In [38]:
np.matmul(zk,Wmat[0])

array([[ -4.05695944,   6.5525545 ,   3.03342369, -13.36467761,
         -1.66519992,   4.73895702,  -5.45961353,  -7.60520738]])

In [45]:

model_input = zk.reshape(1,4)

sum1 = np.matmul(zk,Wmat[0])
sum1bias = sum1 + Wmat[1]
#neuron_out = swish_eval(sum1bias)
neuron_out = sum1bias/(1+np.exp(-sum1bias)) #explicitly writing swish activation

sum2 = np.matmul(neuron_out,Wmat[2])
sum2bias = sum2 + Wmat[3]
neuron_out = sum2bias/(1+np.exp(-sum2bias)) #explicitly writing swish activation

sum3 = np.matmul(neuron_out,Wmat[4])
sum3bias = sum3 + Wmat[5]
neuron_out = sum3bias/(1+np.exp(-sum3bias)) #explicitly writing swish activation

sum4 = np.matmul(neuron_out,Wmat[6])
sum4bias = sum4 + Wmat[7]
neuron_out = sum4bias #sum3bias/(1+np.exp(-sum3bias)) #explicitly writing swish activation


In [46]:
Wmat.shape[0]

12

In [47]:
print(Wmat[1],sum1,sum1bias)
print(neuron_out)
print(neuron_out.shape)

[ 0.17453043 -0.11173429 -0.06356671 -0.02668981  0.48467213 -0.17731553
  0.077495    0.06343799] [[ -4.05695944   6.5525545    3.03342369 -13.36467761  -1.66519992
    4.73895702  -5.45961353  -7.60520738]] [[ -3.88242901   6.44082021   2.96985698 -13.39136741  -1.18052779
    4.56164149  -5.38211853  -7.54176939]]
[[ 0.46356564  0.25528455 -0.27647782 -0.6047259   0.0404524   0.26475612
  -0.24407229 -0.06790939]]
(1, 8)


In [49]:
# print(zk[0,1:4] + neuron_out)

In [50]:
xkplusone_neural = DropModel.predict(zk.reshape(1,4))
print(xkplusone_neural)

[[ 0.2139787  14.415604    0.05232467]]


In [51]:
n_hidden_layers = 3
n_nodes = 4
model_input = zk.reshape(1,4)
sums = np.zeros((n_hidden_layers+1,1))
sumbias = np.zeros((n_hidden_layers+1,1))
Wmat_shape = Wmat.shape[0]

#Initialize from input 
sum0 = np.matmul(zk,Wmat[0])
sumbias0 = sum0 + Wmat[1]
#neuron_out = swish_eval(sum1bias)
neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation
print(neuron_out.shape)


for k in range(n_hidden_layers-1):
    sum0 = np.matmul(neuron_out,Wmat[2*k+2])
    sumbias0 = sum0 + Wmat[2*k+3]
    neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation

#Final output layer with linear activation
sum0 = np.matmul(neuron_out,Wmat[Wmat_shape-2])
sumbias0 = sum0 + Wmat[Wmat_shape-1]
neuron_out = sumbias0  #linear activation


(1, 8)


In [52]:
print(zk[0,1:4] + neuron_out)

[[ 0.02333087 15.34091188  0.51121238]]


In [53]:
xkplusone_neural = DropModel.predict(zk.reshape(1,4))
print(xkplusone_neural)

[[ 0.2139787  14.415604    0.05232467]]


In [54]:
def predictive_model_resnet(zk,Wmat): 
    #Assuming you can pass extra arguments to the function for casadi, otherwsise
    #Number of hidden layers and nodes are given once when we get the optimal resnet
    #The Bayesian Optimization Loop will be changing 
    n_hidden_layers = 5 
    n_nodes = 8
    model_input = zk.reshape(1,4)
    Wmat_shape = Wmat.shape[0]
    
    #Initialize from input 
    sum0 = np.matmul(zk,Wmat[0]) #Matrix multiplication weights*inputs
    sumbias0 = sum0 + Wmat[1] #Add bias to construct y = sum(wi*xi) + b
    #neuron_out = swish_eval(sum1bias)
    neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation, out = fact(y)
    #They are vector quantitites so you might need to change stuff a bit with symbolic vars
    print(neuron_out.shape)

    #Forward prediction in hidden layers
    for k in range(n_hidden_layers-1):
        sum0 = np.matmul(neuron_out,Wmat[2*k+2])
        sumbias0 = sum0 + Wmat[2*k+3]
        neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation

    #Final output layer with linear activation
    sum0 = np.matmul(neuron_out,Wmat[Wmat_shape-2])
    sumbias0 = sum0 + Wmat[Wmat_shape-1]
    neuron_out = sumbias0  #linear activation

    return zk[0,1:4] + neuron_out

In [55]:
zkplusone = predictive_model_resnet(zk,Wmat)
print(zkplusone)
xkplusone_neural = DropModel.predict(zk.reshape(1,4))
print(xkplusone_neural)

(1, 8)
[[ 0.21397871 14.4156034   0.05232468]]
[[ 0.2139787  14.415604    0.05232467]]


Point-wise predictions for MPC
Ignore from here on

In [56]:
def eulerPlantNext(z):
    #next states for plant
    #Define constants
    F = z[0]
    Cx = z[1]
    Ca = z[2]
    Cp = z[3]
    print(F,Ca,Cx,Cp)
    Tsamp=0.005;
    mumax =40
    Ks =2e-1
    Yx =0.04
    Yp =0.8
    pstar=1.8
    Ca0=10.0

    kd= 1e-2;
    aa = 5.5;
    bb = 8e-1;
    
        
    if Cp>pstar:
       mu = 0
    else:
       mu = mumax*(Ca/(Ca+Ks))*(1-Cp/pstar)**3

        

    #Rate Definitions
    fcx=(mu-kd)*Cx - F*Cx
    fcp=aa*fcx +bb*Cx- F*Cp
    fca=-(1/Yx)*fcx - (1/Yp)*fcp + F*(Ca0-Ca)
   
    Cx_kplusone = Cx + Tsamp*fcx
    Ca_kplusone = Ca + Tsamp*fca
    Cp_kplusone = Cp + Tsamp*fcp
    
    
    return Cx_kplusone, Ca_kplusone,Cp_kplusone

In [57]:
uk=np.array([10.0]) #Control Input
xk=np.array([0.2,15,0]) #Current States
zk=np.concatenate((uk,xk)) #

In [58]:
xkplusone_neural = DropModel.predict(zk.reshape(1,4))
print('Xkplusone from residual net:',xkplusone_neural)
xkplusone_plant  = eulerPlantNext(zk)
print('Xkplusone from plant:',xkplusone_plant)


Xkplusone from residual net: [[ 0.2139787  14.415604    0.05232467]]
10.0 15.0 0.2 0.0
Xkplusone from plant: (0.22946368421052632, 13.809845065789474, 0.16285026315789472)


In [ ]:
from casadi import *
